In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammedjunayed/wm811k-silicon-wafer-map-dataset-image")

print("Path to dataset files:", path)

100%|██████████| 983k/983k [00:00<00:00, 94.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/muhammedjunayed/wm811k-silicon-wafer-map-dataset-image/versions/1


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("qingyi/wm811k-wafer-map")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'wm811k-wafer-map' dataset.
Path to dataset files: /kaggle/input/wm811k-wafer-map


In [6]:
DATASET_PATH = "/root/.cache/kagglehub/datasets/muhammedjunayed/wm811k-silicon-wafer-map-dataset-image/versions/1/WM811k_Dataset"

import os
print(os.listdir(DATASET_PATH))


['random', 'none', 'Edge Local', 'Local', 'near full', 'Scratch', 'Center', 'Edge Ring', 'Donut']


In [9]:
import os

BASE_DIR = "/content/wafer_dataset"
splits = ["train", "val", "test"]
classes = ["normal", "defect"]

for s in splits:
    for c in classes:
        os.makedirs(os.path.join(BASE_DIR, s, c), exist_ok=True)

print(" Target folder structure created")


✅ Target folder structure created


In [10]:
import os
import random
import shutil

SOURCE_DIR = "/root/.cache/kagglehub/datasets/muhammedjunayed/wm811k-silicon-wafer-map-dataset-image/versions/1/WM811k_Dataset"
TARGET_DIR = "/content/wafer_dataset"

TRAIN = 0.7
VAL = 0.15

def split_copy(files, class_name):
    random.shuffle(files)
    n = len(files)
    t1 = int(n * TRAIN)
    t2 = int(n * (TRAIN + VAL))

    splits = {
        "train": files[:t1],
        "val": files[t1:t2],
        "test": files[t2:]
    }

    for split_name, imgs in splits.items():
        destination_dir = os.path.join(TARGET_DIR, split_name, class_name)
        # The directories should already exist from the previous cell, but this adds robustness
        os.makedirs(destination_dir, exist_ok=True)
        for img in imgs:
            shutil.copy(img, destination_dir)

# ---------- NORMAL ----------
normal_dir = os.path.join(SOURCE_DIR, "none")
normal_imgs = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir)]
split_copy(normal_imgs, "normal")

# ---------- DEFECT ----------
defect_classes = [d for d in os.listdir(SOURCE_DIR) if d != "none"]

all_defect_imgs = []
for d in defect_classes:
    dpath = os.path.join(SOURCE_DIR, d)
    all_defect_imgs.extend(
        [os.path.join(dpath, f) for f in os.listdir(dpath)]
    )

split_copy(all_defect_imgs, "defect")

print("Dataset split & merged successfully")

✅ Dataset split & merged successfully


In [11]:
for s in ["train", "val", "test"]:
    for c in ["normal", "defect"]:
        path = f"/content/wafer_dataset/{s}/{c}"
        print(s, c, len(os.listdir(path)))


train normal 70
train defect 561
val normal 15
val defect 120
test normal 15
test defect 121


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import os


In [13]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [14]:
DATA_DIR = "/content/wafer_dataset"

train_data = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transform)
val_data   = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_test_transform)
test_data  = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=val_test_transform)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

class_names = train_data.classes
print("Classes:", class_names)


Classes: ['defect', 'normal']


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 2)
)

model = model.to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [17]:
EPOCHS = 10

best_val_acc = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.3f} | Val Acc: {val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "wafer_defect_model.pth")


Epoch 1/10 | Train Loss: 7.300 | Val Acc: 0.889
Epoch 2/10 | Train Loss: 6.158 | Val Acc: 0.889
Epoch 3/10 | Train Loss: 5.802 | Val Acc: 0.889
Epoch 4/10 | Train Loss: 5.341 | Val Acc: 0.889
Epoch 5/10 | Train Loss: 5.076 | Val Acc: 0.896
Epoch 6/10 | Train Loss: 4.713 | Val Acc: 0.874
Epoch 7/10 | Train Loss: 4.575 | Val Acc: 0.896
Epoch 8/10 | Train Loss: 4.303 | Val Acc: 0.904
Epoch 9/10 | Train Loss: 3.906 | Val Acc: 0.896
Epoch 10/10 | Train Loss: 3.952 | Val Acc: 0.919


In [18]:
model.load_state_dict(torch.load("wafer_defect_model.pth"))
model.eval()

correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)


Test Accuracy: 0.8897058823529411


In [22]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names))


[[120   1]
 [ 14   1]]
              precision    recall  f1-score   support

      defect       0.90      0.99      0.94       121
      normal       0.50      0.07      0.12        15

    accuracy                           0.89       136
   macro avg       0.70      0.53      0.53       136
weighted avg       0.85      0.89      0.85       136



In [24]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class names (VERY IMPORTANT: same order as training)
class_names = ['defect', 'normal']

# Recreate model architecture
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 2)
)

# Load trained weights
model.load_state_dict(torch.load("wafer_defect_model.pth", map_location=device))
model = model.to(device)
model.eval()

print("Model loaded successfully")


Model loaded successfully


In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [26]:
def predict_new_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1)
        confidence, pred = torch.max(probs, 1)

    return class_names[pred.item()], confidence.item()


In [30]:
label, confidence = predict_new_image("/content/ChatGPT Image Feb 10, 2026, 12_24_02 AM.png")
print(f"Prediction: {label} | Confidence: {confidence:.2f}")


Prediction: defect | Confidence: 1.00
